---
title: 9.1 Applications
subject:  Symmetric Matrices
subtitle: 
short_title: 9.1 Applications
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: 
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/08_Ch_9_Symmetric_Matrices/102-Apps.ipynb)

{doc}`Lecture notes <../lecture_notes/Lecture 17 - Introduction to Graph Theory and Consensus Protocols.pdf>`

## Reading

Material related to this page, as well as additional exercises, can be

## Learning Objectives

By the end of this page, you should know:
- 

\section{Introduction to Graph Theory and Consensus}

Many applications in engineering involve coordinating groups of agents to behave in a desirable way. Examples abound in a wide range of areas:
\begin{itemize}
    \item Transportation: air traffic control and intelligent transportation systems
    \item Military: distributed operative imaging and battle space management
    \item Robotics: coordination and control among swarms (e.g., on Mars/Moon)
    \item Sensor networks: adaptive ocean sampling, building sensor networks in green buildings
    \item General networks: communication, power, and supply chain networks
\end{itemize}

Many, but not all, of these problems can be posed as consensus problems. In this case, we assume there are $n$ agents, with each agent's state $x_i(t) \in \mathbb{R}$ evolving according to the following differential equation:

\begin{equation}
\dot{x}_i = f_i(x_i) + \sum_{j \in N_i} u_i^j(x_i, x_j), \quad i = 1,\ldots,n \tag{*}
\end{equation}

In equation (*):
\begin{itemize}
    \item the function $f_i: \mathbb{R} \to \mathbb{R}$ defines the dynamics of agent $i$, i.e., how the internal state of agent $i$ evolves in the absence of other agents.
    \item the neighbor set $N_i$ of agent $i$ are those agents that are directly connected to agent $i$, as specified by a connection graph $G$ with nodes $i = 1,\ldots,n$ corresponding to agents, and edges defining inter-agent communication.
    \item the coupling functions $u_i^j: \mathbb{R} \times \mathbb{R} \to \mathbb{R}$ define how the state of agent $j$ influences agent $i$'s dynamics, so that the collective state of the agents $x(t) := (x_1(t),\ldots,x_n(t))$ converges to a desired goal state $x^*$.
\end{itemize}

In many settings of interest, each agent is a regulator that is trying to regulate or estimate a shared value. This is a common model adopted in distributed optimization and learning applications (e.g., to compute an average of sensor measurements or estimate a parameter). In these settings, it makes sense to set the local dynamics function $f_i(x) = 0$, so that an agent's state is determined solely by interactions with its neighbors. Agent $i$'s dynamics then become

\begin{equation}
\dot{x}_i = \sum_{j \in N_i} u_i^j(x_i, x_j), \quad i = 1,\ldots,n \tag{**}
\end{equation}

A common goal for such systems is for all agents to converge to the same value, i.e., that eventually $x_1 = x_2 = \cdots = x_n = m$, for some value $m$. If this happens, system (**) is said to achieve consensus with consensus value $m$.

Studying the behavior of system (**) will require us to bring together all of the ideas that we've seen over the past few lectures on eigenvalues, dynamical systems, and spectral decomposition of symmetric matrices. We start with a summary of relevant facts from graph theory, which will help us model the flow of information in (**).

\section{A Primer on Graph Theory}

We have used graphs before, but we pause now to describe formal linear algebraic representations thereof.

We begin with the basic definition of a graph, which is defined as a pair $G = (V, E)$ that consists of a set of vertices $V$ and a set of edges $E \subseteq V \times V$. A vertex $v \in V$ is a node in the graph, and an edge $e = (v_i, v_j) \in E$ is an arc connecting node $i$ to node $j$. (Note: sometimes this notation is reversed.)

\begin{example}
\begin{tikzpicture}[scale=0.7]
\node[draw, circle] (1) at (0,2) {1};
\node[draw, circle] (2) at (-1,0) {2};
\node[draw, circle] (3) at (0,-2) {3};
\node[draw, circle] (4) at (2,-2) {4};
\node[draw, circle] (5) at (3,0) {5};
\node[draw, circle] (6) at (2,2) {6};

\draw[->] (1) -- (6);
\draw[->] (2) -- (1);
\draw[<->] (2) -- (6);
\draw[->] (3) -- (2);
\draw[->] (3) -- (4);
\draw[->] (4) -- (6);
\draw[->] (5) -- (4);
\draw[->] (5) -- (6);
\draw[->] (6) -- (1);
\end{tikzpicture}

$V = \{1, 2, 3, 4, 5, 6\}$

$E = \{(1,6), (2,1), (2,3), (2,6), (6,2), (3,4),$ 
    $(3,6), (4,3), (4,5), (5,1), (6,1), (6,2), (6,4)\}$

Path $\pi$ with $V_\pi = \{v_5, v_1, v_2, v_6\}$ and 
$E_\pi = \{(v_5,v_1), (v_1,v_2), (v_2,v_6)\}$ (see defn below)
\end{example}

Some important notation and terminology for graphs include:
\begin{itemize}
\item The order of a graph is the number of nodes $|V|$
\item Nodes $v_i$ and $v_j$ are adjacent if there exists $e = (v_i, v_j) \in E$ (i.e., if node $i$ is directly connected to node $j$ via an edge $e$)
\item An adjacent node $v_j$ for a node $v_i$ is called a neighbor of $v_i$
\item The set of all neighbors of $v_i$ is denoted $N_i$
\item A graph $G$ is called complete if all nodes are adjacent
\end{itemize}

We will focus our study on the class of undirected graphs, that is graphs satisfying the property that if $e_{ij} \in E$ then $e_{ji} \in E$. In words, this means that all edges are bidirectional: if $v_j$ is a neighbor of $v_i$, then $v_i$ is a neighbor of $v_j$. Although we won't focus on them, a graph that is not undirected is called directed.

A fundamental question associated with the study of the consensus system (**) is under what properties of interaction among agents can we guarantee that consensus is achieved. Intuitively, if each agent "talks to enough" other agents, we expect consensus to happen. This intuition can be formalized through the notion of graph connectivity.

\subsection{Connectedness of Graphs}

We start by defining a path in $G$, which is a subgraph $\pi = (V_\pi, E_\pi)$, with with distinct nodes $V_\pi = \{v_1, v_2, ..., v_m\}$ and $E_\pi = \{(v_1,v_2),(v_2,v_3),...,(v_{m-1},v_m)\}$. The length of a path $\pi$ is defined as $|E_\pi| = m-1$. For example, in the graph above, the path $\pi$ defined by $V_\pi = \{v_5, v_1, v_6, v_2\}$ and $E_\pi = \{(v_5,v_1),(v_1,v_6),(v_6,v_2)\}$ is the path that goes from node 5 to node 1 to node 6 to node 2. It has length 3, the number of edges traversed. An undirected graph $G$ is called connected if there exists a path $\pi$ between any two distinct nodes of $G$.

\section{A Primer on Graph Theory}

We have used graphs before, but we pause now to describe formal linear algebraic representations thereof.

We begin with the basic definition of a graph, which is defined as a pair $G = (V, E)$ that consists of a set of vertices $V$ and a set of edges $E \subseteq V \times V$. A vertex $v \in V$ is a node in the graph, and an edge $e = (v_i, v_j) \in E$ is an arc connecting node $i$ to node $j$. (Note: sometimes this notation is reversed.)

\begin{example}
\begin{tikzpicture}[scale=0.7]
\node[draw, circle] (1) at (0,2) {1};
\node[draw, circle] (2) at (-1,0) {2};
\node[draw, circle] (3) at (0,-2) {3};
\node[draw, circle] (4) at (2,-2) {4};
\node[draw, circle] (5) at (3,0) {5};
\node[draw, circle] (6) at (2,2) {6};

\draw[->] (1) -- (6);
\draw[->] (2) -- (1);
\draw[<->] (2) -- (6);
\draw[->] (3) -- (2);
\draw[->] (3) -- (4);
\draw[->] (4) -- (6);
\draw[->] (5) -- (4);
\draw[->] (5) -- (6);
\draw[->] (6) -- (1);
\end{tikzpicture}

$V = \{1, 2, 3, 4, 5, 6\}$

$E = \{(1,6), (2,1), (2,3), (2,6), (6,2), (3,4),$ 
    $(3,6), (4,3), (4,5), (5,1), (6,1), (6,2), (6,4)\}$

Path $\pi$ with $V_\pi = \{v_5, v_1, v_2, v_6\}$ and 
$E_\pi = \{(v_5,v_1), (v_1,v_2), (v_2,v_6)\}$ (see defn below)
\end{example}

Some important notation and terminology for graphs include:
\begin{itemize}
\item The order of a graph is the number of nodes $|V|$
\item Nodes $v_i$ and $v_j$ are adjacent if there exists $e = (v_i, v_j) \in E$ (i.e., if node $i$ is directly connected to node $j$ via an edge $e$)
\item An adjacent node $v_j$ for a node $v_i$ is called a neighbor of $v_i$
\item The set of all neighbors of $v_i$ is denoted $N_i$
\item A graph $G$ is called complete if all nodes are adjacent
\end{itemize}

We will focus our study on the class of undirected graphs, that is graphs satisfying the property that if $e_{ij} \in E$ then $e_{ji} \in E$. In words, this means that all edges are bidirectional: if $v_j$ is a neighbor of $v_i$, then $v_i$ is a neighbor of $v_j$. Although we won't focus on them, a graph that is not undirected is called directed.

A fundamental question associated with the study of the consensus system (**) is under what properties of interaction among agents can we guarantee that consensus is achieved. Intuitively, if each agent "talks to enough" other agents, we expect consensus to happen. This intuition can be formalized through the notion of graph connectivity.

\subsection{Connectedness of Graphs}

We start by defining a path in $G$, which is a subgraph $\pi = (V_\pi, E_\pi)$, with with distinct nodes $V_\pi = \{v_1, v_2, ..., v_m\}$ and $E_\pi = \{(v_1,v_2),(v_2,v_3),...,(v_{m-1},v_m)\}$. The length of a path $\pi$ is defined as $|E_\pi| = m-1$. For example, in the graph above, the path $\pi$ defined by $V_\pi = \{v_5, v_1, v_6, v_2\}$ and $E_\pi = \{(v_5,v_1),(v_1,v_6),(v_6,v_2)\}$ is the path that goes from node 5 to node 1 to node 6 to node 2. It has length 3, the number of edges traversed. An undirected graph $G$ is called connected if there exists a path $\pi$ between any two distinct nodes of $G$.

\section{Example: These graphs are connected (all edges are bidirectional).}

\begin{tikzpicture}[scale=0.5]
\begin{scope}
    \node[draw, circle] (1) at (0,0) {};
    \node[draw, circle] (2) at (-1,1) {};
    \node[draw, circle] (3) at (1,1) {};
    \node[draw, circle] (4) at (-1,-1) {};
    \node[draw, circle] (5) at (1,-1) {};
    \draw (1) -- (2) -- (3) -- (1) -- (4) -- (5) -- (1);
\end{scope}

\begin{scope}[xshift=5cm]
    \node[draw, circle] (1) at (0,2) {};
    \node[draw, circle] (2) at (-1,1) {};
    \node[draw, circle] (3) at (1,1) {};
    \node[draw, circle] (4) at (0,0) {};
    \node[draw, circle] (5) at (-1,-1) {};
    \node[draw, circle] (6) at (1,-1) {};
    \draw (1) -- (2) -- (4) -- (3) -- (1);
    \draw (4) -- (5);
    \draw (4) -- (6);
\end{scope}
\end{tikzpicture}

This graph, composed of two disconnected subgraphs, is not:

\begin{tikzpicture}[scale=0.5]
\node[draw, circle] (1) at (0,0) {};
\node[draw, circle] (2) at (1,1) {};
\node[draw, circle] (3) at (1,-1) {};
\node[draw, circle] (4) at (3,0) {};
\draw (1) -- (2) -- (3) -- (1);
\end{tikzpicture}

\section{Matrices Associated with a Graph}

In our study of network flow problems, we encountered the incidence matrix associated with a graph. However, for our consensus problems, we can associate another matrix with a graph $G$, and while the incidence matrix is convenient for encoding flow conservation, we will see that the Graph Laplacian Matrix is more natural for defining consensus algorithms. We begin by defining the adjacency matrix $A \in \mathbb{R}^{n \times n}$ of graph $G$ of order $n$ by

\[
a_{ij} = \begin{cases}
1 & \text{if } (v_i, v_j) \in E, \quad i,j=1,\ldots,n \\
0 & \text{otherwise}
\end{cases}
\]

The degree matrix $\Delta$ of a graph $G$ is a diagonal $n \times n$ matrix with diagonal elements $\Delta_{ii}$ specified by the number of edges leaving node $i$, also called the out-degree of $v_i$, which we'll denote by out$(v_i)$:

\[
\Delta_{ii} = \text{out}(v_i), \quad i=1,\ldots,n
\]

The Laplacian matrix $L$ of a graph is defined as $L = \Delta - A$. An important property of the Laplacian is that its rows all sum to zero, and that if a graph is undirected, then its adjacency matrix and its Laplacian are both symmetric.

Example: The adjacency matrix and Laplacian for the directed graph above are:

\[
A = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 1 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 \\
1 & 1 & 0 & 1 & 0 & 0
\end{bmatrix}
\quad
L = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & -1 \\
0 & 2 & 0 & -1 & 0 & -1 \\
0 & 0 & 2 & -1 & 0 & -1 \\
0 & 0 & -1 & 2 & -1 & 0 \\
-1 & 0 & 0 & 0 & 1 & 0 \\
-1 & -1 & 0 & -1 & 0 & 3
\end{bmatrix}
\]

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath=/08_Ch_9_Symmetric_Matrices/102-Apps.ipynb)